# A continuation from `2-pup-model-explore.ipynb`, here we load the model and evaluate it on some never before seen URLs to see how well it classifies them.

In [12]:
from feature_extraction import process_url_data
import json
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import re
import sys
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from urllib.parse import urlparse
import xgboost as xgb
from xgboost import plot_importance

Load the model from `5-pup-model-explore.ipynb`

In [13]:
gBm = joblib.load('./models-checkpoints/xgb-model.pkl') # load the model

Now that we've loaded the model, let's process some unknown data (just randomly pulled from openphish.org)

In [18]:
df_final = process_url_data('./datasets/20240412-openphish.csv')

This should return df_final

In [19]:
df_final.head()

,Lexical_domain,Lexical_domainNoPrefix,Lexical_domainLength,Lexical_pathLength,Lexical_queryLength,Lexical_numPathComponents,Lexical_numQueryComponents,Lexical_hasDigitsInDomain,Lexical_hasDigitsInPath,Lexical_hasDigitsInQuery,Descriptive_domainLength,Descriptive_pathLength,Descriptive_queryLength,Descriptive_numPathComponents,Descriptive_filename,Descriptive_fileNamePresent,Descriptive_fileExtension,Descriptive_isIpAddress,Descriptive_fileExecutable
11,33,35,43,12,0,1,0,True,False,False,43,12,0,2,73,1,3,False,False
15,14,14,11,29,0,4,0,False,False,False,11,29,0,5,32,1,3,False,False
16,14,14,11,52,0,7,0,False,False,False,11,52,0,8,32,1,3,False,False
17,14,14,11,75,0,10,0,False,False,False,11,75,0,11,32,1,3,False,False
21,77,80,42,22,52,1,1,True,True,True,42,22,52,2,27,1,3,False,False


In [20]:
df_final.describe()

,Lexical_domain,Lexical_domainNoPrefix,Lexical_domainLength,Lexical_pathLength,Lexical_queryLength,Lexical_numPathComponents,Lexical_numQueryComponents,Descriptive_domainLength,Descriptive_pathLength,Descriptive_queryLength,Descriptive_numPathComponents,Descriptive_filename,Descriptive_fileNamePresent,Descriptive_fileExtension
count,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.0,107.000000
mean,38.915888,38.915888,34.962617,24.588785,27.299065,1.813084,0.289720,34.850467,24.588785,27.299065,2.813084,36.757009,1.0,3.719626
std,27.206104,27.206104,13.150023,21.866643,67.150663,1.415086,0.739828,13.306123,21.866643,67.150663,1.415086,21.154348,0.0,1.951267
min,0.000000,0.000000,10.000000,8.000000,0.000000,1.000000,0.000000,10.000000,8.000000,0.000000,2.000000,0.000000,1.0,0.000000
25%,11.500000,11.500000,21.000000,11.500000,0.000000,1.000000,0.000000,20.000000,11.500000,0.000000,2.000000,20.500000,1.0,3.000000
50%,36.000000,36.000000,43.000000,15.000000,0.000000,1.000000,0.000000,43.000000,15.000000,0.000000,2.000000,37.000000,1.0,3.000000
75%,62.500000,62.500000,43.000000,35.000000,0.000000,2.000000,0.000000,43.000000,35.000000,0.000000,3.000000,52.500000,1.0,3.000000
max,89.000000,89.000000,76.000000,157.000000,304.000000,10.000000,5.000000,76.000000,157.000000,304.000000,11.000000,74.000000,1.0,10.000000


In [ ]:
if y.dtype == 'object' or y.dtype.name == 'category':
    le = LabelEncoder()
    y = le.fit_transform(y)

# split training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)